In [151]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from IPython.display import clear_output

clear_output()

In [152]:
df = pd.read_pickle('data/saeb_pb_filtered.pkl')
df.head()

,ano,sigla_uf,escola_publica,id_aluno,situacao_censo,disciplina,amostra,proficiencia,erro_padrao,proficiencia_saeb,...,responsaveis_incentivam_comparecer_aulas,tempo_chegada_escola,forma_chegada_escola,transporte_escolar,tempo_lazer,tempo_cursos,tempo_estudos,leitura_noticias,leitura_livros_geral,leitura_historia_quadrinhos
0,2021,PB,1,49369547,1,CH,0,-1.076008,0.479786,189.482456,...,.,.,.,.,.,.,.,None,None,None
1,2021,PB,0,48935836,1,CH,1,0.212093,0.483460,264.988420,...,C,B,C,B,D,C,B,None,None,None
2,2021,PB,1,49370798,1,CH,0,-1.093155,0.665270,188.477352,...,.,.,.,.,.,.,.,None,None,None
3,2021,PB,1,49163192,1,CH,0,-2.013929,0.610384,134.503387,...,.,C,F,B,.,.,A,None,None,None
4,2021,PB,1,49355938,1,CH,0,-1.622314,0.677914,157.459089,...,.,.,.,.,.,.,.,None,None,None


In [153]:
'''
df1 = pd.read_pickle('data/saeb_pb_2017.pkl')
df1.head()

df1['preenchimento_caderno'].value_counts(dropna=False)

# filter by 'preenchimento_caderno' = 1 and 'presenca' = 1
print(len(df1))
df1 = df1[df1['preenchimento_caderno'] == 1]
print(len(df1)) 

#select columns with dtype object
df1.select_dtypes(include=['object'])


df.drop(columns=['situacao_censo'], inplace=True)

# concat df1 onto df using only df columns
df = pd.concat([df, df1[df.columns]], ignore_index=True)
df['ano'].unique()

'''

"\ndf1 = pd.read_pickle('data/saeb_pb_2017.pkl')\ndf1.head()\n\ndf1['preenchimento_caderno'].value_counts(dropna=False)\n\n# filter by 'preenchimento_caderno' = 1 and 'presenca' = 1\nprint(len(df1))\ndf1 = df1[df1['preenchimento_caderno'] == 1]\nprint(len(df1)) \n\n#select columns with dtype object\ndf1.select_dtypes(include=['object'])\n\n\ndf.drop(columns=['situacao_censo'], inplace=True)\n\n# concat df1 onto df using only df columns\ndf = pd.concat([df, df1[df.columns]], ignore_index=True)\ndf['ano'].unique()\n\n"

In [154]:
# selecting categorical columns
columns_to_filter = ['possui_eletrecidade', 'possui_escrivaninha', 'escolaridade_mae', 'escolaridade_pai', 'responsaveis_leem', 'responsaveis_incentivam_realizacao_licao_casa', 'responsaveis_incentivam_estudos', 'responsaveis_incentivam_comparecer_aulas', 'tempo_chegada_escola', 'forma_chegada_escola', 'transporte_escolar', 'tempo_lazer', 'tempo_cursos', 'tempo_estudos']

# remove rows with all values '.' or '*'
df = df[(df[columns_to_filter] != '.').all(axis=1)]
df = df[(df[columns_to_filter] != '*').all(axis=1)]
print(len(df))

df.head()

118745


,ano,sigla_uf,escola_publica,id_aluno,situacao_censo,disciplina,amostra,proficiencia,erro_padrao,proficiencia_saeb,...,responsaveis_incentivam_comparecer_aulas,tempo_chegada_escola,forma_chegada_escola,transporte_escolar,tempo_lazer,tempo_cursos,tempo_estudos,leitura_noticias,leitura_livros_geral,leitura_historia_quadrinhos
1,2021,PB,0,48935836,1,CH,1,0.212093,0.483460,264.988420,...,C,B,C,B,D,C,B,None,None,None
17,2021,PB,0,48935834,1,CH,1,-1.302525,0.700937,176.204509,...,C,A,F,B,C,A,B,None,None,None
63,2021,PB,1,49164452,1,CH,0,-0.207503,0.511880,240.392529,...,C,C,A,B,A,C,C,None,None,None
64,2021,PB,1,49368109,1,CH,0,-0.765889,0.611475,207.661042,...,C,A,A,A,D,C,D,None,None,None
65,2021,PB,1,49353247,1,CH,0,-0.686009,0.559540,212.343416,...,C,A,D,B,A,A,C,None,None,None


In [155]:
# print columns with dtype object
print(df.select_dtypes(include=['object']).columns)

Index(['sigla_uf', 'id_aluno', 'disciplina', 'desempenho_aluno', 'sexo',
       'raca_cor', 'possui_eletrecidade', 'possui_escrivaninha',
       'escolaridade_mae', 'escolaridade_pai', 'responsaveis_leem',
       'responsaveis_incentivam_realizacao_licao_casa',
       'responsaveis_incentivam_estudos',
       'responsaveis_incentivam_comparecer_aulas', 'tempo_chegada_escola',
       'forma_chegada_escola', 'transporte_escolar', 'tempo_lazer',
       'tempo_cursos', 'tempo_estudos', 'leitura_noticias',
       'leitura_livros_geral', 'leitura_historia_quadrinhos'],
      dtype='object')


In [156]:
df['idade'].value_counts(dropna=False)

idade
<NA>    61830
15      25475
14      20293
16       7224
17       2899
2021     1024
Name: count, dtype: Int64

In [157]:
# nulling '2021' from 'idade'
df['idade'] = df['idade'].apply(lambda x: x if x != 2021 else np.nan)


# change 'idade' type to float (temporarily)
df['idade'] = df['idade'].astype(float)

# fill 'idade' missing values with the mean (int)
df.fillna({'idade': df['idade'].mean()}, inplace=True)
df['idade'] = df['idade'].astype(int)

df['idade'].head()

1     15
17    14
63    15
64    16
65    16
Name: idade, dtype: int64

In [158]:
print(df['idade'].value_counts())

print('\n')

print(df['idade'].describe())

idade
14    83147
15    25475
16     7224
17     2899
Name: count, dtype: int64


count    118745.000000
mean         14.409449
std           0.714114
min          14.000000
25%          14.000000
50%          14.000000
75%          15.000000
max          17.000000
Name: idade, dtype: float64


In [159]:
df['desempenho_aluno'].value_counts(dropna=False)

desempenho_aluno
Básico          60283
Insuficiente    26051
Proficiente     21071
None             7636
Avançado         3704
Name: count, dtype: int64

In [160]:
print(df['proficiencia_saeb'].isnull().sum())

# drop null values from 'proficiencia_saeb'
df.dropna(subset=['proficiencia_saeb'], inplace=True)

print(df['proficiencia_saeb'].isnull().sum())

2
0


In [161]:
# changing 'desempenho_aluno' to categorical numerical values
# Insuficiente = 0, Básico = 1, Proficiente = 2, Avançado = 3
# dropping None values

# drop the ones with value 'None'
_ = df.dropna(subset=['desempenho_aluno']).copy()

_.loc[:, 'desempenho_aluno_cat'] = _['desempenho_aluno'].map({'Insuficiente': 0, 'Básico': 1, 'Proficiente': 2, 'Avançado': 3})
#df['desempenho_aluno'] = df['desempenho_aluno'].astype(int)

df = _

df['desempenho_aluno_cat'].value_counts(dropna=False)

desempenho_aluno_cat
1    60283
0    26051
2    21071
3     3704
Name: count, dtype: int64

In [162]:
for column in columns_to_filter:
    print(column, df[column].unique())

possui_eletrecidade ['B' 'A']
possui_escrivaninha ['A' 'B']
escolaridade_mae ['C' 'A' 'F' 'E' 'D' 'B']
escolaridade_pai ['A' 'D' 'F' 'C' 'E' 'B']
responsaveis_leem ['A' 'C' 'B' None]
responsaveis_incentivam_realizacao_licao_casa ['C' 'B' 'A']
responsaveis_incentivam_estudos ['C' 'B' 'A']
responsaveis_incentivam_comparecer_aulas ['C' 'A' 'B']
tempo_chegada_escola ['A' 'B' 'C']
forma_chegada_escola ['A' 'D' 'C' 'B' 'F' 'G' 'E']
transporte_escolar ['A' 'B' None]
tempo_lazer ['C' 'A' 'D' 'B']
tempo_cursos ['C' 'A' 'D' 'B']
tempo_estudos ['B' 'C' 'D' 'A']


In [163]:
# questão "tempo_chegada_escola"
# Quanto tempo você demora para chegar à sua escola?
# A: Menos de 30 minutos (0)
# B: Entre 30 minutos e uma hora (1)
# C: Mais de uma hora (2)
df['tempo_chegada_escola'] = df['tempo_chegada_escola'].map({'A': 0, 'B': 1, 'C': 2})

df['tempo_chegada_escola'].value_counts(dropna=False)

# OBS.: não será utilizado pois tem dados de 2021 (pandemia)

tempo_chegada_escola
0    85778
1    21225
2     4106
Name: count, dtype: int64

In [164]:
# questões "tempo_"
# Fora da escola em dias de aula, quanto tempo você usa para: 
# A: Não uso meu tempo para isso (0)
# B: Menos de 1 hora (1)
# C: Entre 1 a 2 horas (2)
# D: Mais de 2 horas (3)

tempo_columns = ['tempo_lazer', 'tempo_cursos', 'tempo_estudos']

for column in tempo_columns:
    df[column] = df[column].map({'A': 0, 'B': 1, 'C': 2, 'D': 3})

df[tempo_columns].head()

,tempo_lazer,tempo_cursos,tempo_estudos
69,2,2,1
70,2,2,1
71,0,0,2
126,3,0,2
127,2,0,3


In [165]:
# questões "responsaveis_"
# Com que frequência seus pais ou responsáveis costumam:
# A: Nunca ou Quase Nunca
# B: Raramente
# C: Sempre ou Quase Sempre
# 0, 1, 2

responsaveis = ['responsaveis_incentivam_realizacao_licao_casa', 'responsaveis_incentivam_estudos', 'responsaveis_incentivam_comparecer_aulas']

for column in responsaveis:
    df[column] = df[column].map({'A': 0, 'B': 1, 'C': 2})

df[responsaveis].head()

,responsaveis_incentivam_realizacao_licao_casa,responsaveis_incentivam_estudos,responsaveis_incentivam_comparecer_aulas
69,2,2,2
70,2,2,2
71,2,2,2
126,2,2,2
127,2,2,2


In [166]:
df['escolaridade_mae'].value_counts(dropna=False)

escolaridade_mae
F    30412
D    25007
B    15742
C    14950
A    14473
E    10525
Name: count, dtype: int64

In [167]:
# questões "escolaridade_" e "forma_chegada_escola"
# usar onehotencoder pois são categóricas nominais

from sklearn.preprocessing import OneHotEncoder

columns_to_onehot = ['escolaridade_mae', 'escolaridade_pai', 'forma_chegada_escola']

for column in columns_to_onehot:
    encoder = OneHotEncoder(sparse_output=False)
    encoded = encoder.fit_transform(df[column].values.reshape(-1, 1))
    encoded = pd.DataFrame(encoded, columns=[f'{column}_{i}' for i in range(encoded.shape[1])])
    df = pd.concat([df, encoded], axis=1)
    df.drop(column, axis=1, inplace=True)

df.head()

,ano,sigla_uf,escola_publica,id_aluno,situacao_censo,disciplina,amostra,proficiencia,erro_padrao,proficiencia_saeb,...,escolaridade_pai_5,escolaridade_pai_6,forma_chegada_escola_0,forma_chegada_escola_1,forma_chegada_escola_2,forma_chegada_escola_3,forma_chegada_escola_4,forma_chegada_escola_5,forma_chegada_escola_6,forma_chegada_escola_7
69,2021,PB,1,43575034,1,MT,0,-0.220310,0.350673,237.650742,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
70,2021,PB,1,43575034,1,LP,0,-1.053385,0.447439,191.950468,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
71,2021,PB,1,49370848,1,LP,0,-0.785038,0.389307,206.734607,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
126,2021,PB,1,43517493,1,MT,0,-0.357253,0.377772,229.996679,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
127,2021,PB,1,43509591,1,MT,0,1.000746,0.363297,305.898405,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [168]:
len(df)

155841

In [169]:
print(df['possui_eletrecidade'].value_counts(dropna=False))
# sugere-se que 'B' significa 'Sim' e 'A' significa 'Não'

df['possui_eletrecidade'] = df['possui_eletrecidade'].map({'A': 0, 'B': 1})

df['possui_eletrecidade'].value_counts()

possui_eletrecidade
B      101462
NaN     44732
A        9647
Name: count, dtype: int64


possui_eletrecidade
1.0    101462
0.0      9647
Name: count, dtype: int64

In [170]:
df['possui_escrivaninha'].value_counts()
# seguindo a lógica do anterior, 'B' significa 'Sim' e 'A' significa 'Não'

df['possui_escrivaninha'] = df['possui_escrivaninha'].map({'A': 0, 'B': 1})

df['possui_escrivaninha'].value_counts()

possui_escrivaninha
0.0    57767
1.0    53342
Name: count, dtype: int64

In [171]:
#responsaveis_leem ['A' 'B' 'C' None]
#transporte_escolar ['A' 'B' None]

df['responsaveis_leem'] = df['responsaveis_leem'].map({'A': 0, 'B': 1, 'C': 2})
df['transporte_escolar'] = df['transporte_escolar'].map({'A': 0, 'B': 1})

In [172]:
save = True

if save:
    df.to_pickle('data/saeb_treated.pkl')